
<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/3_Prepare_Data_Partition_LabelMap_CSVtoXML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/3_Prepare_Data_Partition_LabelMap_CSVtoXML.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>


### Prepare dataset ###

#### Connect to Google Drive

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Copy dataset from Google Drive and extract


In [31]:
%pwd
%cd /content/drive/My Drive/Datasets/
%pwd
%cd /content
!pwd

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-31-96163274b413>", line 1, in <module>
    get_ipython().magic('pwd')
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2160, in magic
    return self.run_line_magic(magic_name, magic_arg_s)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2081, in run_line_magic
    result = fn(*args,**kwargs)
  File "<decorator-gen-90>", line 2, in pwd
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/magic.py", line 188, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/magics/osm.py", line 244, in pwd
    return py3compat.getcwd()
FileNotFoundError: [Errno 2] No such file or directory

During handling of the above exception, another excepti

FileNotFoundError: ignored

In [28]:
#%cd /content/drive/My Drive/Datasets/
#%cp unmasked-dataset.zip /content/
#%cp masked-dataset.zip /content/
%cd /content/
%mkdir -p training_demo/images
%cd training_demo/images
%pwd

#!unzip -j unmasked-dataset.zip 
#!unzip -j masked-dataset.zip 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-3b91118710cf>", line 4, in <module>
    get_ipython().magic('cd /content/')
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2160, in magic
    return self.run_line_magic(magic_name, magic_arg_s)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2081, in run_line_magic
    result = fn(*args,**kwargs)
  File "<decorator-gen-91>", line 2, in cd
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/magic.py", line 188, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/magics/osm.py", line 288, in cd
    oldcwd = py3compat.getcwd()
FileNotFoundError: [Errno 2] No such file or directory

During handling of the above exception, anothe

FileNotFoundError: ignored

## Partitioning the images

We continue our data preparation by splitting our dataset into training and test sets.

Typically, the ratio is 90%/10%, i.e. 90% of the images are used for training and the rest 10% is maintained for testing, but you can chose whatever ratio suits your needs.


Lets add our dataset into our notebook (optional)

### Import necessary libraries

In [0]:
import os
import re
import shutil
from PIL import Image
import shutil
import glob
import math
import random
import xml.etree.ElementTree as ET

#### Define folder locations and split ratio

In [8]:
imageDir = os.getcwd() + "/training_demo/images"
print(imageDir)
outputDir = imageDir 
ratio = 0.1

/content/training_demo/images


### Partitioning the images

#### Helper Code

In [9]:
def iterate_dir(source, dest, ratio):
    source = source.replace('\\', '/')
    dest = dest.replace('\\', '/')
    train_dir = os.path.join(dest, 'train')
    print(train_dir)
    test_dir = os.path.join(dest, 'test')
    print(test_dir)
    
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)

        
    images = [f for f in os.listdir(source)
              if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(.jpg|.jpeg|.png)$', f)]

     
    num_images = len(images)
    num_test_images = math.ceil(ratio*num_images)
    
    print(num_test_images)
        
    for i in range(num_test_images):
        idx = random.randint(0, len(images)-1)
        
        filename = images[idx]
        filename_path = os.path.join(source, filename)
        
        xml_filename = os.path.splitext(filename)[0]+'.xml'
        xml_path = os.path.join(source, xml_filename)
        
        #copy image + XML files across only if accompanying XML file exists
        if os.path.isfile(xml_path):     
                copyfile(filename_path, os.path.join(test_dir, filename))
                copyfile(xml_path,os.path.join(test_dir,xml_filename))
        else:
            print("WARNING: the following XML file does not exist:" + xml_path)
        images.remove(images[idx])

    for filename in images:
        xml_filename = os.path.splitext(filename)[0]+'.xml'
        xml_path = os.path.join(source, xml_filename)
        filename_path = os.path.join(source, filename)
        
        #copy image + XML files across only if accompanying XML file exists
        if os.path.isfile(xml_path):
                copyfile(filename_path, os.path.join(train_dir, filename))
                copyfile(xml_path, os.path.join(train_dir, xml_filename))
        else:
            print("WARNING: the following XML file does not exist:" + xml_path)

iterate_dir(imageDir, outputDir, ratio)

/content/training_demo/images/train
/content/training_demo/images/test
0


#### Execute code to partition the images

NameError: ignored

#### Delete original files (Optional) 

Once we are happy the images have successfully split into training and test sets, we can go ahead and delete the original images.

In [30]:
!rm -v /content/training_demo/images/*

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
removed '/content/content/training_demo/images/0004STET6P.jpg'
removed '/content/content/training_demo/images/0004STET6P.xml'
removed '/content/content/training_demo/images/000N7AIAFT.jpg'
removed '/content/content/training_demo/images/000N7AIAFT.xml'
removed '/content/content/training_demo/images/00H858UYSD.jpg'
removed '/content/content/training_demo/images/00H858UYSD.xml'
removed '/content/content/training_demo/images/00KPGHV40E.jpg'
removed '/content/content/training_demo/images/00KPGHV40E.xml'
removed '/content/content/training_demo/images/00P2LUXJW3.jpg'
removed '/content/content/training_demo/images/00P2LUXJW3.xml'
removed '/content/content/training_demo/images/00PYA83V1P.jpg'
removed '/content/content/training_demo/images/00PYA83V1P.xml'
removed '/content/content/training_demo/images/00SD82OK2A.jpg'
removed '/content/content/training_demo/images/00SD82OK2A.xml'
re

## Label Map

TensorFlow requires a label map, which namely maps each of the used labels to an integer values. This label map is used both by the training and detection processes.

In [0]:
!mkdir -p training_demo/annotations
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/label_map.pbtxt -P training_demo/annotations

--2020-06-03 17:45:18--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/label_map.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79 [text/plain]
Saving to: ‘training_demo/annotations/label_map.pbtxt’

label_map.pbtxt     100%[===================>]      79  --.-KB/s    in 0s      

2020-06-03 17:45:19 (7.06 MB/s) - ‘training_demo/annotations/label_map.pbtxt’ saved [79/79]



## XML to CSV ##

And finally we need to generate a CSV file containing all image detail / classes from the individual XML files.

### Define locations for data

In [0]:
dataset_path = os.getcwd() + "/training_demo"
images_path = dataset_path + "/images"
annotations_path = dataset_path + "/annotations"


### Code to generate CSV files

In [0]:
def xml_to_csv(path):
    xml_list = []
    print("test1"+path)
    for xml_file in glob.glob(path + '/*.xml'):
        print(xml_file)
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
            
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

def generate_csv(folder):
    xml_path = os.path.join(images_path , folder)
    print(xml_path)
    xml_df = xml_to_csv(xml_path)
    print(xml_df)
    xml_df.to_csv(annotations_path + '/' + folder +'_labels.csv', index=None)
    csv_path = dataset_path + "/" + folder
    print('Successfully converted train xml to csv.')

#### Generate CSV files

For the train and test folders and place result in annotations folder

In [0]:
generate_csv("train")
generate_csv("test")